https://dacon.io/competitions/official/236178/data

**[설명]**

4일 후의 프로젝트를 위해 준비된 이 데이터셋에는 인간이 작성한 리뷰와 인공지능이 작성한 리뷰가 섞여 있습니다.

하지만 어떤 리뷰가 인간에 의해 작성되었는지를 나타내는 레이블 대부분이 사라져버렸습니다.

여러분의 임무는 일부 레이블이 남아있는 학습용 데이터셋을 활용하여,

테스트 데이터셋의 네 개 리뷰 중 어떤 것이 실제 인간에 의해 작성된 것인지 정확하게 예측하는 것입니다!

당신의 통찰력을 활용하여 테스트 데이터셋의 'label' 필드를 복구해주세요!

* train.csv [파일]
* id : 샘플 고유 id
* sentence1 : 리뷰 텍스트 1
* sentence2 : 리뷰 텍스트 2
* sentence3 : 리뷰 텍스트 3
* sentence4 : 리뷰 텍스트 4
* label : 사람이 작성한 원본 리뷰 텍스트 번호; [1, 2, 3, 4] 중 하나

In [1]:
# drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install sentencepiece
!pip install -U transformers sentencepiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.8 MB/s eta 0:00:00


In [3]:
# # install cupf, cupy, cuml
# !pip install pyproject
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py

# Module

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler

from keras.utils import pad_sequences

from transformers import AdamW
from transformers import AutoTokenizer, AutoModel

In [5]:
train_df = pd.read_csv(r'/content/drive/MyDrive/Share/Personal/NLP/data/train.csv')
test_df = pd.read_csv(r'/content/drive/MyDrive/Share/Personal/NLP/data/test.csv')

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         50 non-null     object
 1   sentence1  50 non-null     object
 2   sentence2  50 non-null     object
 3   sentence3  50 non-null     object
 4   sentence4  50 non-null     object
 5   label      50 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 2.5+ KB


# Data Preprocessing

In [7]:
train_df

,id,sentence1,sentence2,sentence3,sentence4,label
0,TRAIN_000,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것...,직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서...,직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 ...,2
1,TRAIN_001,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,분위기가 너무 좋아요! 2층 창문이 넓어서 쾌적한 느낌이에요. 조명도 아름답고 음료...,분위기가 짱!! 2층 창문이 커서 탁 트여있는 느낌이에요 ㅎㅎ 조명도 예쁘고 음료랑...,분위기가 너무 좋아요! 2층 창문이 크고 넓어서 탁 트여있는 느낌이에요. 조명도 예...,3
2,TRAIN_002,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",일단 장사가 잘되길 바라는 마음에서 별5개 드립니다 간도 맞았고 매운걸 좋아하는 입...,일단 저는 장사가 잘되기를 바라는 마음에서 별 다섯 개를 주고 싶어요. 맛도 딱 맞...,"먼저, 칭찬과 응원의 의미로 별 다섯 개를 주고 싶습니다. 간도 딱 맞고, 저는 매...",2
3,TRAIN_003,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...","1편의 신선함에 비해 약간 빛이 바래 보이지만, 여전히 재미있게 즐길 수 있어요. ...","1편의 독특함 때문에 약간의 비교가 불가피하지만, 이 게임은 여전히 흥미로워요. 시...",1편이 워낙 참신했던 탓에 좀 묻힌 감이 있긴 하지만 재미는 여전합니다. 시스템도 ...,4
4,TRAIN_004,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",빵점을 주고 싶지만 참아냈습니다... 이 영화는 사상 유래가 없는 것 같아요. 조지...,빵점 주고 싶을 정도로 엄청 실망했어요... 이 영화는 별들의 전쟁처럼 역사적인 작...,"빵점을 주고 싶었는데 참았어요... 이런 영화는 전례가 없는데, 조지 루카스의 스타...",1
5,TRAIN_005,"치킨 반마리 주문 가능. 골뱅이소면과 함께 추천. 치킨은 평범, 사이드메뉴와 함께하...",둘이서 치킨 반마리 주문이 가능해서 골뱅이소면무침랑 같이 시킬 수 있었네요! 치킨맛...,"두 사람이서 반마리의 치킨을 주문할 수 있어서, 골뱅이소면무침을 함께 주문했습니다....",두 사람이서 치킨 반마리 주문이 가능해서 골뱅이소면무침과 함께 먹을 수 있었습니다....,2
6,TRAIN_006,진짜 일하시는 사람들 너무 시끄러워요 자기들 개인사얘기하는데 목소리도 크고 밥이 ...,진짜 시끄러운데... 이런거 시끄러워서 돈 아까워.,진짜 이 직장은 너무 시끄럽네요. 직원들끼리만 얘기하는게 아니라 목소리도 크고 밥먹...,"진짜 일하는 분들이 너무 시끄러웠어요. 개인적인 이야기를 할 때도 목소리가 크고, ...",1
7,TRAIN_007,"우와 정말 멋진 스터디 카페입니다. 다른 지점은 가보지 않았지만, 노트북을 많이 사...","와우, 정말 좋아요! 이 스터디 카페는 정말 최고예요. 다른 지점은 가보지 않았지만...","와우, 정말 좋은 공부 카페예요. 다른 지점은 가본 적은 없지만, 노트북을 많이 사...",우와 완전좋아요 스터디카페 신세계 ... 다른지점은 안가봤지만 노트북많이쓰는데 노트...,4
8,TRAIN_008,가시려는 분들은 이전 리뷰를 잘 읽어보고 가는 것을 추천합니다. 급하게 여기에서 상...,가시고자 하시는 분들은 앞서 쓴 후기 잘 읽어보고 가세요. 급해서 여기서 상견례했는...,이 식당을 방문하실 분들께는 앞선 리뷰들을 잘 읽어보시고 결정하시는 것을 추천드립니...,요즘은 리뷰를 믿기 힘들죠. 하지만 제 경험을 공유하고자 해요. 급하게 가서 상견례...,2
9,TRAIN_009,라운즈 앱에서 안경 구매 후 도수렌즈 구매했어요. 제가 눈이 나쁘고 렌즈 결정하기가...,라운즈 앱을 통해 안경과 도수 렌즈를 구입했습니다. 저의 시력이 좋지 않아서 렌즈 ...,라운즈 앱을 통해 안경과 도수렌즈를 구매하였습니다. 제 시력이 좋지 않아 렌즈 결정...,라운즈 앱으로 안경 구매 및 도수렌즈 선택. 눈 상태 때문에 까다롭지만 직원 친절함...,1


클래스 균형을 위해 ai 가 작성한 sentence 중 하나만 random으로 choice
- 1개의 data point에 대해서 1개는 인간, 3개는 ai 데이터
- 결과적으로 1:3의 클래스 비율
- 1:1로 맞춰주는 작업 수행

In [8]:
human_reviews = []  # 정답 data
ai_reviews = [] # 오답 data

In [9]:
for i, label in enumerate(train_df['label']):
    labels = [1, 2 ,3 ,4]
    labels.remove(label)
    ai_label = np.random.choice(labels)
        # 정답 label을 제외한 ai label 중에서 임의로 1가지 선택
        # 1가지만 남기기 위함
    human_reviews.append(train_df.iloc[i, label])
    ai_reviews.append(train_df.iloc[i, ai_label])

In [10]:
print(f'human_reviews : {len(human_reviews)}')
print(f'ai_reviews : {len(ai_reviews)}')

human_reviews : 50
ai_reviews : 50


In [11]:
human_labels = [1] * len(human_reviews)  # 정답 레이블
ai_labels = [0] * len(ai_reviews)   # 오답 레이블

In [12]:
reviews = human_reviews + ai_reviews
labels = human_labels + ai_labels

print(f'reviews : {len(reviews)}')
print(f'labels : {len(labels)}')

reviews : 100
labels : 100


# Tokenizer

In [13]:
# KoGPT-2 사용
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [14]:
# GPT Classifier Model
class GPTModel(nn.Module):
    def __init__(self, num_classes=2):
        super(GPTModel, self).__init__()
        self.num_classes = num_classes
        self.gpt = AutoModel.from_pretrained('skt/kogpt2-base-v2')
        self.fc = nn.Linear(768, num_classes, bias=True)

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.gpt(input_ids=input_ids,
                          attention_mask=attention_mask)
        output = self.fc(output[0])
        output = output[torch.arange(self.num_classes, device=device), -1]
        return output

In [15]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [16]:
model = GPTModel()
model = model.to(device)

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [17]:
model

GPTModel(
  (gpt): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (fc): Linear(in_features=768, out_features=2, bias=True)
)

# DataLoader

In [18]:
tokenized_texts = [tokenizer.tokenize(s) for s in reviews]
len(tokenized_texts)

100

In [19]:
# padding을 위한 max_len 확인
max_len = max([len(x) for x in tokenized_texts])

# token의 vectorization
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# padding 추가
input_ids = pad_sequences(input_ids, maxlen=max_len, dtype='long',
                         truncating='post', padding='post')
    # dypte : 정수형(long)
    # truncating : 시퀀스 끝에서부터 데이터 편집 (시퀀스 길이가 maxlen보다 긴 경우)
    # padding : 0 추가

In [20]:
attention_mask = []

for seq in tqdm(input_ids):
    seq_mask = [float(i>0) for i in seq]
        # seq의 각 원소가 패딩이 아닌 경우 1.0으로 표현
    attention_mask.append(seq_mask)

train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(labels)
train_masks = torch.tensor(attention_mask)

print(f'train_inputs.shape : {train_inputs.shape}')
print(f'train_labels.shape : {train_labels.shape}')
print(f'train_masks.shape : {train_masks.shape}')

100%|██████████| 100/100 [00:00<00:00, 9981.45it/s]

train_inputs.shape : torch.Size([100, 125])
train_labels.shape : torch.Size([100])
train_masks.shape : torch.Size([100, 125])


# Training Setting

In [21]:
# hyper parameter
batch_size = 2
epochs = 10
lr = 2e-5
eps = 1e-8

In [22]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [23]:
optimizer = AdamW(model.parameters(), lr=lr, eps=eps)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [24]:
# Accuracy Metric
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Train

In [25]:
def train(dataloader, criterion, epochs):
    for epoch in range(epochs):
        print('--------------------------------------------------------------------')
        print(f'Epoch "{epoch + 1}"')

        print('Train Mode:', end=" ")
        total_loss, train_accuracy = 0.0, 0.0
        model.train()

        for batch in tqdm(dataloader):
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            outputs = model(input_ids=b_input_ids,
                            attention_mask=b_input_mask)

            loss = criterion(outputs.view(-1, 2), b_labels)
            logits = outputs.view(-1, 2)
            total_loss += loss.item()
            loss.backward()

            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            train_accuracy += flat_accuracy(logits, label_ids)

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        model.zero_grad()
        avg_train_loss = total_loss / len(dataloader)
        avg_train_acc = train_accuracy / len(dataloader)
        print(" Average training loss: {0:.2f}".format(avg_train_loss), end=" ")
        print(" Average training accuracy: {0:.2f}".format(avg_train_acc))


In [26]:
train(train_dataloader, criterion, epochs)

--------------------------------------------------------------------
Epoch "1"
Train Mode: 

100%|██████████| 50/50 [00:08<00:00,  5.75it/s]


 Average training loss: 0.99  Average training accuracy: 0.75
--------------------------------------------------------------------
Epoch "2"
Train Mode: 

100%|██████████| 50/50 [00:05<00:00,  9.17it/s]


 Average training loss: 1.05  Average training accuracy: 0.74
--------------------------------------------------------------------
Epoch "3"
Train Mode: 

...

 Average training loss: 0.45  Average training accuracy: 0.94
--------------------------------------------------------------------
Epoch "9"
Train Mode: 

100%|██████████| 50/50 [00:04<00:00, 10.27it/s]


 Average training loss: 0.07  Average training accuracy: 0.99
--------------------------------------------------------------------
Epoch "10"
Train Mode: 

100%|██████████| 50/50 [00:04<00:00, 10.74it/s]

 Average training loss: 0.59  Average training accuracy: 0.94


In [27]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Share/Personal/NLP/project/best_model.pth')

# Predict

- test 데이터도 train 데이터와 동일한 프로세스로 처리
- train 데이터와 달리 test 데이터는 4개의 문장 중 가장 인간이 작성했을 법한 문장을 예측하는 것이기 때문에 순서대로 배치
- 4개의 문장 중 가장 score가 높은 2개 선택

In [29]:
test_reviews = []

for i in range(test_df.shape[0]):
  cols = [1,2,3,4]
  test_reviews.extend(test_df.iloc[i, cols])

In [31]:
tokenized_texts = [tokenizer.tokenize(s) for s in test_reviews]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=max_len, dtype='long', truncating='post', padding='post')

attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [32]:
model.eval()

preds = []

for batch in tqdm(test_dataloader):
  batch = tuple(t.to(device) for t in batch) # Batch to cuda device
  b_input_ids, b_input_mask = batch
  outputs = model(input_ids=b_input_ids,
                  attention_mask=b_input_mask)

  logits = outputs.view(-1, 2).detach().cpu().numpy()
  preds.extend(logits)

preds = np.array(preds)

100%|██████████| 2200/2200 [00:57<00:00, 38.18it/s]


In [33]:
pred_labels = []

for i in range(0,preds.shape[0],4):
  tmp = preds[i:i+4,1] # 4 문장씩 검사
  label1 = np.argmax(tmp) # 가장 높은 score
  tmp[label1] = -np.inf
  label2 = np.argmax(tmp) # 두번째로 높은 score

  label = str(label1 + 1) + str(label2 + 1)
  pred_labels.append(label)

# Submission

In [34]:
submit = pd.read_csv('/content/drive/MyDrive/Share/Personal/NLP/data/sample_submission.csv')
submit['label'] = pred_labels
submit

,id,label
0,TEST_0000,34
1,TEST_0001,13
2,TEST_0002,24
3,TEST_0003,21
4,TEST_0004,32
...,...,...
1095,TEST_1095,21
1096,TEST_1096,24
1097,TEST_1097,41
1098,TEST_1098,41


In [35]:
submit.to_csv('/content/drive/MyDrive/Share/Personal/NLP/project/binary_clf_gpt_submission.csv', index=False)